#### Symbolic Propagation fingerprint

SPF stands for Symbolic Propagation fingerprint, described [here](Symbolic_propagation_fingerprint.md)

first we need to calibrate the SPF model, by measuring the signal received in each room of the building. This is done by walking in each room, and measuring the signal received by the CartoTag. The CartoTag is held above the head, to limit the absorption of the signal by the body of the wearer. The wearer of the CartoTag moves within the room, which allows to have a representation of the absorption for the whole room and not only for the position at which the wearer stayed.

the calibration part will be done in an app and using a proper database in the future if the model is efficient enough.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from API import API
from secret import API_KEY,MAC_WEAR
import plotly.graph_objects as go

from func.DataLoader import gdfLoader,ReadMarkerMap,RSSIDataLoader


In [2]:
#Calibration time setting

start,end="2023-06-22 10:40:00.000000","2023-06-22 11:30:35.000000"
time={
    "21855":("2023-06-22 10:41:00.000000","2023-06-22 10:44:00.000000"),#4A435
    "21279":("2023-06-22 10:46:00.000000","2023-06-22 10:49:00.000000"),#couloir
    "21267":("2023-06-22 10:51:00.000000","2023-06-22 10:54:00.000000"),#ascenseur
    "21821":("2023-06-22 10:56:00.000000","2023-06-22 11:03:00.000000"),#4A436
    "21820":("2023-06-22 11:06:00.000000","2023-06-22 11:09:00.000000"),#4A438
}

In [3]:
ModuleMapPath="data/carteModulePalaiseau.csv"
marker_gdf=ReadMarkerMap(ModuleMapPath)
marker_gdf

,macModule,geometry
0,A8032A31204E,POINT (2.20079 48.71316)
1,C45BBE39F42A,POINT (2.20110 48.71315)
2,C45BBE39F9AE,POINT (2.20120 48.71314)
3,A8032A311FAA,POINT (2.20072 48.71317)
4,C45BBE39F56A,POINT (2.20117 48.71318)
5,C45BBE37B346,POINT (2.20087 48.71315)
6,C45BBE386952,POINT (2.20098 48.71316)
7,C45BBE386362,POINT (2.20094 48.71314)
8,C45BBE384CD2,POINT (2.20093 48.71317)
9,C45BBE39870E,POINT (2.20106 48.71315)


In [4]:
# data gathering
Gatien_API=API(API_KEY)
hour_correction=2
start=pd.to_datetime(start)-pd.Timedelta(hour_correction, unit="h")
end=pd.to_datetime(end)-pd.Timedelta(hour_correction, unit="h")
data=Gatien_API.getRawDataForCartoWear('C77C2F92664E',start,end)
data=pd.DataFrame(data)
data["timestamp"]=pd.to_datetime(data['timestamp'])



In [5]:
room_data={}
for room in time:
    room_data[room]=data[(data['timestamp'] > pd.to_datetime(time[room][0])-pd.Timedelta(hour_correction, unit="h")) & (data['timestamp'] < pd.to_datetime(time[room][1])-pd.Timedelta(hour_correction, unit="h"))]
    room_data[room]=room_data[room].reset_index(drop=True)
print(room_data)

{'21855':                   timestamp     macModule  rssi
0   2023-06-22 08:43:59.561  A8032A311F96   -71
1   2023-06-22 08:43:59.217  A8032A311F6A   -54
2   2023-06-22 08:43:58.002  A8032A311DAA   -77
3   2023-06-22 08:43:56.570  A8032A311F6A   -61
4   2023-06-22 08:43:56.132  A8032A31204A   -80
..                      ...           ...   ...
146 2023-06-22 08:41:04.837  A8032A311F96   -77
147 2023-06-22 08:41:04.103  A8032A311F6A   -58
148 2023-06-22 08:41:02.289  A8032A311F96   -76
149 2023-06-22 08:41:01.440  A8032A311F6A   -61
150 2023-06-22 08:41:00.802  A8032A311DAA   -77

[151 rows x 3 columns], '21279':                   timestamp     macModule  rssi
0   2023-06-22 08:48:59.296  A8032A311F6A   -76
1   2023-06-22 08:48:59.260  A8032A311F96   -65
2   2023-06-22 08:48:58.124  A8032A311DAA   -75
3   2023-06-22 08:48:56.711  A8032A311F96   -63
4   2023-06-22 08:48:56.636  A8032A311F6A   -58
..                      ...           ...   ...
334 2023-06-22 08:46:04.121  A8032A31204A   

In [6]:
outlier_suppress=0.35
#define macmodule expected rssi value for each room
macmodule_expected_rssi={}
for room in room_data:
    macmodule_expected_rssi[room]={}
    # for the room x i received signal from the macmodule list
    macmodule_list=room_data[room].macModule.unique()
    for macmodule in macmodule_list:
        # for each macmodule i received signal from the room x list
        rssi_list_for_macmodule=room_data[room][room_data[room].macModule==macmodule].rssi
        macmodule_expected_rssi[room][macmodule]=(rssi_list_for_macmodule.quantile(outlier_suppress),rssi_list_for_macmodule.quantile(1-outlier_suppress))
macmodule_expected_rssi    


{'21855': {'A8032A311F96': (-74.0, -70.05),
  'A8032A311F6A': (-59.8, -57.0),
  'A8032A311DAA': (-75.0, -72.6),
  'A8032A31204A': (-80.0, -78.0),
  'A8032A30FBFE': (-80.0, -79.7)},
 '21279': {'A8032A311F6A': (-62.0, -59.0),
  'A8032A311F96': (-65.0, -60.0),
  'A8032A311DAA': (-73.0, -70.0),
  'C45BBE386362': (-79.0, -77.95),
  'A8032A31204A': (-75.0, -73.25),
  'C45BBE38FEAA': (-79.0, -79.0),
  'C45BBE39870E': (-79.0, -78.0),
  'C45BBE37B346': (-76.1, -73.0),
  'A8032A312052': (-79.0, -75.0),
  'A8032A30FBFE': (-80.0, -77.55),
  'C45BBE39F4D6': (-79.0, -78.0),
  'C45BBE386952': (-80.0, -79.0),
  'C45BBE384CD2': (-78.9, -78.0)},
 '21267': {'A8032A311F6A': (-72.0, -69.0),
  'A8032A30FBFE': (-75.0, -71.0),
  'A8032A311F96': (-75.0, -71.0),
  'A8032A311DAA': (-62.8, -58.0),
  'A8032A31204A': (-80.0, -78.0),
  'C45BBE39870E': (-78.0, -78.0),
  'C45BBE38FEAA': (-79.65, -79.35),
  'C45BBE39F4D6': (-80.0, -80.0),
  'C45BBE384CD2': (-78.0, -77.4),
  'C45BBE386362': (-77.6, -76.7),
  'C45BBE37B3

In [7]:
macmodule_list=marker_gdf.macModule.tolist()
macmodule_expected_room={}
for macModule in macmodule_list:
    macmodule_expected_room[macModule]={}
    for room in macmodule_expected_rssi:
        macmodule_expected_room[macModule][room]={}
        if macModule in macmodule_expected_rssi[room]:
            macmodule_expected_room[macModule][room]=macmodule_expected_rssi[room][macModule]
macmodule_expected_room


{'A8032A31204E': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE39F42A': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE39F9AE': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'A8032A311FAA': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': (-69.45, -67.0)},
 'C45BBE39F56A': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE37B346': {'21855': {},
  '21279': (-76.1, -73.0),
  '21267': (-76.85, -74.15),
  '21821': (-79.0, -79.0),
  '21820': (-79.6, -79.0)},
 'C45BBE386952': {'21855': {},
  '21279': (-80.0, -79.0),
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE386362': {'21855': {},
  '21279': (-79.0, -77.95),
  '21267': (-77.6, -76.7),
  '21821': {},
  '21820': {}},
 'C45BBE384CD2': {'21855': {},
  '21279': (-78.9, -78.0),
  '21267': (-78.0, -77.4),
  '21821': {},
  '21820': {}},
 'C45BBE39870E': {'21855': {},
  '21279': (-79.0

In [8]:
import json
with open("data/spf_calibration/calib1.json", "w") as file:
    json.dump(macmodule_expected_room, file)
    file.close()